In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root
from ROOT import TChain

In [ ]:
def open_df(fname,treename="flashtrigger/opflash_opflash_tree"):
    return pd.DataFrame( root2array(fname, treename) )

In [ ]:
def get_eff(df,param,xmin,xmax,interval,total):
    if total <= 0:
        print 'Cannot calculate efficiency w/o total <= 0!'
        raise Exception
    underflow = float(len(df.query('%s <=0' % param).groupby(['run','subrun','event'])))
    xaxis = np.arange(xmin,xmax,interval)

    eff = np.zeros(len(xaxis))
    for index in xrange(len(xaxis)):
        cut_value = xaxis[index]
        ctr = 0
        ctr = underflow + float(len(df.query('%s > %g' % (param,cut_value)).groupby(['run','subrun','event'])))
        eff[index] = ctr / total

    return (xaxis,eff)

In [ ]:
#
# Data loading
#
df1 = open_df('flash_trigger_ana/cosmic.root')
df2 = open_df('flash_trigger_ana/data_cosmic.root')
#df3 = open_df('flash_trigger_ana/bnb.root')
#df_v = [df1,df2,df3]
df_v = [df1,df2]
label_v = ['MCC 6.1 Cosmics','Data Cosmics (Run 2377)']
#df_v = [df1]
colors = ['blue','red','magenta']

In [ ]:
cand_df_v=[]
for i in xrange(len(df_v)):
    cand_df_v.append(df_v[i].query('-0<dt and dt<1.6'))
    print '1.6us window event fraction:', 
    print len(cand_df_v[-1].groupby(['run','subrun','event'])),
    print '/',
    print len(df_v[i].groupby(['run','subrun','event']))
param='pe_total'
xmin,xmax,interval = (0,1000,5)

eff_v=[]
xaxis_v=[]
ymin=2
ymax=0
for i in xrange(len(cand_df_v)):
    total = len(df_v[i].groupby(['run','subrun','event']))
    print '>50 PE event fraction...',len(cand_df_v[i].query('pe_total>50').groupby(['run','subrun','event'])),'/',total
    xaxis,eff = get_eff(cand_df_v[i],param,xmin,xmax,interval,total)
    if eff.max() > ymax: ymax=eff.max()
    if eff.min() < ymin: ymin=eff.min()
    eff_v.append(eff)
    xaxis_v.append(xaxis)
    
ybuf = (ymax - ymin) * 0.05
#ymax += ybuf
#ymin -= ybuf
ymax = 0.01
ymin = 0

fig, ax = plt.subplots(figsize=(12, 8))
#ax.set_yscale('log')

plt.ylim(ymin,ymax)
plt.xlim(xmin-interval*2,xmax+interval*2)
for i in xrange(len(eff_v)):
    ax.plot(xaxis_v[i], eff_v[i], marker='o', label=label_v[i],linestyle='--',color=colors[i])
plt.xlabel('Cut Value [p.e.]',fontsize=20)
plt.ylabel('Efficiency',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('Total Photo-Electron (p.e.) Cut Selection Efficiency',fontsize=20)
ax.legend(prop={'size':20})
plt.show()

if (xmax - xmin)/interval > 100:
    fig, ax = plt.subplots(figsize=(12, 8))
#    ax.set_yscale('log')

    ymin = 2
    ymax = 0
    for eff in eff_v:
        if eff[0:50].min() < ymin: ymin = eff[0:50].min()
        if eff[0:50].max() > ymax: ymax = eff[0:50].max()
    ybuf = (ymax - ymin) * 0.05
#    ymax += ybuf
#    ymin -= ybuf
    ymax = 0.01
    ymin = 0
    plt.ylim(ymin,ymax)
    plt.xlim(xmin,xmin + interval * 50)
    for i in xrange(len(eff_v)):
        ax.plot(xaxis_v[i], eff_v[i], marker='o',label=label_v[i], linestyle='--',color=colors[i])
    plt.xlabel('Cut Value [p.e.]',fontsize=20)
    plt.ylabel('Efficiency',fontsize=20)
    plt.tick_params(labelsize=20)
    plt.grid()
    plt.title('Total Photo-Electron (p.e.) Cut Selection Efficiency',fontsize=20)
    ax.legend(prop={'size':20})
    plt.show()
        
#sg_of = len(df.query('%s > %g' % (param,xmax))) / float(total) * 100
#sg_of = int(sg_of*1000.)/1000.
        
#print 'overflow',  sg_of,'[%]
